In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from matplotlib import pyplot as plt
%pylab inline

In [ ]:
d = pd.read_csv('fiverrUsersPic_analyze_with_projectposition_Animation_and_3D.csv', sep='\t', index_col=0)

In [ ]:
d.info()

In [ ]:
d[pd.notnull(d.position_on_screen_high_rating)][:5]['position_on_screen_high_rating']

In [ ]:
def process_dict(dict_str):
    return 10

In [ ]:
d['position_on_screen_high_rating_avg'] = d.position_on_screen_high_rating.apply(process_dict)

In [ ]:
d[:3].T